In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import numpy as np

# Load the dataset from the Excel file
file_path_psp = '../data/raw/PSP_Jan_Feb_2019.xlsx'
file_path_fee = '../data/raw/PSP_Fees.xlsx'
df = pd.read_excel(file_path_psp)
df_fee = pd.read_excel(file_path_fee)

# Define attempts

* Definition:

In [ ]:
# Sort the dataset by 'country', 'amount', 'PSP', and 'tmsp'
df_sorted = df.sort_values(by=['tmsp', 'country', 'amount', 'PSP']).reset_index(drop=True)

# Initialize an empty list to store the 'attempts' values
attempts = [1]  # The first row will always have an attempt value of 1

# Loop through the sorted DataFrame to calculate the 'attempts'
for i in range(1, len(df_sorted)):
    row = df_sorted.iloc[i]
    prev_row = df_sorted.iloc[i-1]

    # Check if 'country', 'amount', and 'PSP' are the same and 'tmsp' is within 60 seconds
    if (row['country'] == prev_row['country'] and
        row['amount'] == prev_row['amount'] and
        row['PSP'] == prev_row['PSP'] and
        (row['tmsp'] - prev_row['tmsp']).seconds < 60):
        attempts.append(attempts[-1] + 1)
    else:
        attempts.append(1)

# Add the 'attempts' column to the DataFrame
df_sorted['attempts'] = attempts

# Show the first few rows
df_sorted.head(10)

In [ ]:
# Group the data by 'country', 'amount', 'PSP', and 'attempts' and count unique attempts
unique_attempts_count = df_sorted.groupby(['country', 'amount', 'PSP'])['attempts'].max().reset_index()
unique_attempts_distribution = unique_attempts_count['attempts'].value_counts().sort_index()

# Plot the distribution of 'attempts'
plt.figure(figsize=(10, 6))
sns.barplot(x=unique_attempts_distribution.index, y=unique_attempts_distribution.values, palette="Blues_d")
plt.title('Verteilung der Versuche (attempts)')
plt.xlabel('Anzahl der Versuche')
plt.ylabel('Häufigkeit')
plt.show()

unique_attempts_distribution

In [ ]:
# Merge the fees DataFrame with the original dataset based on 'PSP' and 'success'
df_merged = pd.merge(df_sorted, fees_df_read, on=['PSP', 'success'], how='left')

# Show the first few rows of the merged DataFrame to verify
df_merged.head()

# Time-related features

In [ ]:
# Extract time-related features
df_merged['weekday'] = df_merged['tmsp'].dt.day_name()
df_merged['hour'] = df_merged['tmsp'].dt.hour

# Create time-of-day feature
bins = [0, 6, 12, 18, 24]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']
df_merged['time_of_day'] = pd.cut(df_merged['hour'], bins=bins, labels=labels, right=False)

# Create month and quarter features
df_merged['month'] = df_merged['tmsp'].dt.month
df_merged['quarter'] = df_merged['tmsp'].dt.quarter

# Create transaction-related features
df_merged['fee_to_amount_ratio'] = df_merged['fee'] / df_merged['amount']
df_merged['log_amount'] = np.log1p(df_merged['amount'])  # log1p is used to handle zero amounts

# Show the first few rows of the DataFrame to verify the new features
df_merged.head()

# Transactional features

In [ ]:
# Create interactive features
df_merged['country_PSP'] = df_merged['country'] + "_" + df_merged['PSP']
df_merged['PSP_3D_secured'] = df_merged['PSP'] + "_" + df_merged['3D_secured'].astype(str)

# Initialize columns for previous successes/failures and transaction duration
df_merged['previous_successes'] = 0
df_merged['previous_failures'] = 0
df_merged['transaction_duration'] = 0

# Loop through the DataFrame to calculate previous successes, previous failures, and transaction duration
# We only consider records that have 'attempts' greater than 1
groups = df_merged[df_merged['attempts'] > 1].groupby(['country', 'amount', 'PSP'])
for name, group in groups:
    success_count = 0
    failure_count = 0
    for i in range(len(group)):
        row = group.iloc[i]
        index = row.name
        if i == 0:
            # For the first record in the group, transaction duration is zero
            df_merged.at[index, 'transaction_duration'] = 0
        else:
            prev_row = group.iloc[i - 1]
            duration = (row['tmsp'] - prev_row['tmsp']).seconds
            df_merged.at[index, 'transaction_duration'] = duration

        if row['success'] == 1:
            success_count += 1
        else:
            failure_count += 1

        # Update the DataFrame with the accumulated counts
        df_merged.at[index, 'previous_successes'] = success_count
        df_merged.at[index, 'previous_failures'] = failure_count

# Show the first few rows of the DataFrame to verify the new features
df_merged[df_merged['attempts'] > 1].head()


In [ ]:
df_merged.to_excel('PSP_Jan_Feb_2019_ready.xlsx', index=False)